## Logistic Regression
Logistic Regression is a Machine Learning classification algorithm that is used to predict the probability of a categorical dependent variable. In logistic regression, the dependent variable is a binary variable that contains data coded as 1 (yes, success, etc.) or 0 (no, failure, etc.). In other words, the logistic regression model predicts P(Y=1) as a function of X.

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from numpy import mean

import pandas as pd

def encode_df(df):
  columnsToEncode = list(df.select_dtypes(include=['object']))
  le = LabelEncoder()
  for feature in columnsToEncode:
      try:
          df[feature] = le.fit_transform(df[feature])
      except:
          print('Error encoding ' + feature)
  return df

X_train_full = pd.read_csv("./preprocessed/mergedTrainData.csv")
X_test_full = pd.read_csv("./preprocessed/mergedTestData.csv")

X_train_full = encode_df(X_train_full)
X_test_full = encode_df(X_test_full)

# Obtain target and predictors
#features = ["years_since_loan","amount","duration","payments","balance","frequency","years_since_acc_open","region","no. of inhabitants","no. of municipalities with inhabitants < 499 ","no. of municipalities with inhabitants 500-1999","no. of municipalities with inhabitants > 2000","no. of cities ","ratio of urban inhabitants ","average salary ","unemployment_rate","no. of enterpreneurs per 1000 inhabitants ","no. of commited crimes","type","gender","age_group"]

features = ["duration","payments","balance","itr_balance_per_account", "frequency","years_since_acc_open","region","ratio entrepeneurs","average salary ","unemploymant_growth","criminality_growth","age_group"]

y_train = X_train_full.status
X_train = X_train_full[features]
X_test = X_test_full[features]

# Define the model
model = LogisticRegression(solver='liblinear', random_state=0, class_weight='balanced')

# Define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

param_grid = {'C': [0.001, 0.01, 0.1, 0.25, 0.5, 1, 2.5, 5, 10],
                  'solver': ['liblinear', 'sag', 'saga'] 
             }
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv,
scoring='roc_auc')

# summarize performance
# print('Mean ROC AUC: %.3f' % mean(scores))

grid_result = grid.fit(X_train, y_train)
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))

# Evaluate the model
p_pred = grid_result.predict_proba(X_test)

resultData = {'Id': X_test_full['loan_id'], 'Predicted': p_pred[:,1]}
result = pd.DataFrame(data=resultData)
result.to_csv("./results/logisticRegression.csv", index=None)

/home/dukes/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/dukes/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/dukes/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/dukes/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/dukes/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/dukes/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: Convergen

Best: 0.703450 using {'C': 0.001, 'solver': 'saga'}


/home/dukes/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/dukes/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/dukes/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


# Random Forest

In [4]:
# https://stackoverflow.com/questions/30814231/using-the-predict-proba-function-of-randomforestclassifier-in-the-safe-and-rig
# https://rpmcruz.github.io/machine%20learning/2018/02/09/probabilities-trees.html

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(bootstrap=False,
                                class_weight='balanced',
                                criterion='gini',
                                max_depth=6,
                                max_features='auto',
                                min_samples_leaf=6,
                                min_samples_split=2,
                                n_estimators=1000,
                                random_state=0)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
param_grid = {
    'max_depth': [6],
    'min_samples_leaf': [6],
    'min_samples_split': [2],
}
grid = GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=-1, cv=cv,
scoring='roc_auc')

grid_result = grid.fit(X_train, y_train)
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))

p_pred = grid_result.predict_proba(X_test)

resultData = {'Id': X_test_full['loan_id'], 'Predicted': p_pred[:,1]}
result = pd.DataFrame(data=resultData)
result.to_csv("./results/randomForest.csv", index=None)

Best: 0.641920 using {'max_depth': 6, 'min_samples_leaf': 6, 'min_samples_split': 2}


# SVM

In [5]:
# https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47

from sklearn.svm import SVC

clf = SVC(probability=True, class_weight='balanced')
clf.fit(X_train, y_train)
p_pred = clf.predict_proba(X_test)

resultData = {'Id': X_test_full['loan_id'], 'Predicted': p_pred[:,1]}
result = pd.DataFrame(data=resultData)
result.to_csv("./results/SVM.csv", index=None)